# Scaling Two-Tower training with Vertex AI

In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
REGION = 'us-central1'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"REGION: {REGION}")

VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com'

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
REGION: us-central1


In [2]:
import os
import json
from datetime import datetime
from time import time
import pandas as pd
# disable INFO and DEBUG logging everywhere
import logging
import time
from pprint import pprint
import pickle as pkl

logging.disable(logging.WARNING)

from google.cloud import aiplatform as vertex_ai
from google.cloud import storage

In [3]:
storage_client = storage.Client(project=PROJECT_ID)

vertex_ai.init(project=PROJECT_ID,location=REGION)

### update vars

In [4]:
VERSION= "trainerv3"
APP='sp'
MODEL_TYPE='2tower'
FRAMEWORK = 'tfrs'
MODEL_ROOT_NAME = f'{APP}-{MODEL_TYPE}-{FRAMEWORK}-{VERSION}'

print(f"MODEL_ROOT_NAME: {MODEL_ROOT_NAME}")

MODEL_ROOT_NAME: sp-2tower-tfrs-trainerv3


## Create training package

In [5]:
REPO_DOCKER_PATH_PREFIX = 'src'

In [6]:
# Docker definitions for training
IMAGE_NAME = f'{MODEL_ROOT_NAME}-tr'
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

DOCKERNAME = 'tfrs'
REPO_DOCKER_PATH_PREFIX = 'src'
MACHINE_TYPE ='e2-highcpu-32'
FILE_LOCATION = './src'

print(f"IMAGE_URI: {IMAGE_URI}")

IMAGE_URI: gcr.io/hybrid-vertex/sp-2tower-tfrs-trainerv3-tr


In [7]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/cloudbuild.yaml

steps:
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', '$_IMAGE_URI', '$_FILE_LOCATION', '-f', '$_FILE_LOCATION/Dockerfile.$_DOCKERNAME']
images:
- '$_IMAGE_URI'

Overwriting src/cloudbuild.yaml


In [8]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/two_tower_jt/train_config.py
MAX_PLAYLIST_LENGTH = 5

Overwriting src/two_tower_jt/train_config.py


In [9]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/two_tower_jt/requirements.txt
google-cloud-aiplatform>=1.20.0
tensorflow-recommenders==0.7.2
tensorboard==2.10.1
tensorboard-data-server==0.6.1
tensorboard-plugin-profile==2.11.1
tensorflow-io==0.27.0
google-cloud-aiplatform[cloud_profiler]>=1.20.0

Overwriting src/two_tower_jt/requirements.txt


In [10]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/Dockerfile.{DOCKERNAME}

# FROM tensorflow/tensorflow:2.10.1-gpu
FROM gcr.io/deeplearning-platform-release/tf-gpu.2-10

WORKDIR /src

# Copies the trainer code to the docker image.
COPY two_tower_jt/* two_tower_jt/ 

RUN pip install -r two_tower_jt/requirements.txt

RUN apt update && apt -y install nvtop

Overwriting src/Dockerfile.tfrs


### Build Training image with Cloud Build

In [11]:
print(f"DOCKERNAME: {DOCKERNAME}")
print(f"IMAGE_URI: {IMAGE_URI}")
print(f"FILE_LOCATION: {FILE_LOCATION}")
print(f"MACHINE_TYPE: {MACHINE_TYPE}")

DOCKERNAME: tfrs
IMAGE_URI: gcr.io/hybrid-vertex/sp-2tower-tfrs-trainerv3-tr
FILE_LOCATION: ./src
MACHINE_TYPE: e2-highcpu-32


In [12]:
! gcloud config set gcloudignore/enabled true

Updated property [gcloudignore/enabled].


In [13]:
%%writefile .gcloudignore
.gcloudignore
/local_files/
/img/
*.pkl
*.png
*.ipynb
.git
.github
.ipynb_checkpoints/*
*__pycache__
*cpython-37.pyc
spotipy_secret_creds.py
custom_pipeline_spec.json
/WIP/*
beam_candidates/*
beam_training/*

Overwriting .gcloudignore


In [14]:
# !gcloud meta list-files-for-upload
# !ls

In [15]:
! gcloud builds submit --config src/cloudbuild.yaml \
    --substitutions _DOCKERNAME=$DOCKERNAME,_IMAGE_URI=$IMAGE_URI,_FILE_LOCATION=$FILE_LOCATION \
    --timeout=2h \
    --machine-type=$MACHINE_TYPE

Creating temporary tarball archive of 40 file(s) totalling 291.6 KiB before compression.
Uploading tarball of [.] to [gs://hybrid-vertex_cloudbuild/source/1679359364.149285-55e263160e954ed1aaf66a81d137dffd.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/hybrid-vertex/locations/global/builds/21e6788f-1db5-4c9b-ad5c-ee317b42e6bc].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/21e6788f-1db5-4c9b-ad5c-ee317b42e6bc?project=934903580331 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "21e6788f-1db5-4c9b-ad5c-ee317b42e6bc"

FETCHSOURCE
Fetching storage object: gs://hybrid-vertex_cloudbuild/source/1679359364.149285-55e263160e954ed1aaf66a81d137dffd.tgz#1679359364533979
Copying gs://hybrid-vertex_cloudbuild/source/1679359364.149285-55e263160e954ed1aaf66a81d137dffd.tgz#1679359364533979...
/ [1 files][ 53.2 KiB/ 53.2 KiB]                                                
Operation completed over 1 objects/53

## Prepare Train Job Specs

In [25]:
VOCAB_GCS ='gs://jt-tfrs-central-v3/customjob-test-trainerv2/run-20230222-225502/vocab_dict.pkl'
!gsutil cp $VOCAB_GCS vocab_dict_jt.pkl

Copying gs://jt-tfrs-central-v3/customjob-test-trainerv2/run-20230222-225502/vocab_dict.pkl...
/ [1 files][ 18.5 MiB/ 18.5 MiB]                                                
Operation completed over 1 objects/18.5 MiB.                                     


In [26]:
filehandler = open('vocab_dict_jt.pkl', 'rb')
vocab_dict = pkl.load(filehandler)
filehandler.close()

In [27]:
for keys in vocab_dict:
    print(keys)

pl_name_src
track_name_pl
artist_name_pl
album_name_pl
artist_genres_pl
tracks_playlist_titles_pl
track_name_can
artist_name_can
album_name_can
artist_genres_can
track_pl_titles_can


### Training Accelerators

In [28]:
WORKER_MACHINE_TYPE = 'a2-highgpu-1g'
REPLICA_COUNT = 1
ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
PER_MACHINE_ACCELERATOR_COUNT = 1
REDUCTION_SERVER_COUNT = 0                                                      
REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"
DISTRIBUTE_STRATEGY = 'single'

## Vertex Experiments

### Create an experiemnt and experiment run

In [29]:
EXPERIMENT_PREFIX = 'customtest5'                     # custom identifier for organizing experiments
EXPERIMENT_NAME=f'{EXPERIMENT_PREFIX}-{VERSION}'
RUN_NAME = f'run-{time.strftime("%Y%m%d-%H%M%S")}'

print(f"EXPERIMENT_NAME: {EXPERIMENT_NAME}")
print(f"RUN_NAME: {RUN_NAME}")

EXPERIMENT_NAME: customtest5-trainerv3
RUN_NAME: run-20230321-005425


### Create Managed TensorBoard

In [30]:
# use existing TB instance
# TB_RESOURCE_NAME = 'projects/934903580331/locations/us-central1/tensorboards/6924469145035603968'

# # create new TB instance
TENSORBOARD_DISPLAY_NAME=f"{EXPERIMENT_NAME}-v1"
tensorboard = vertex_ai.Tensorboard.create(display_name=TENSORBOARD_DISPLAY_NAME, project=PROJECT_ID, location=REGION)
TB_RESOURCE_NAME = tensorboard.resource_name


print(f"TB_RESOURCE_NAME: {TB_RESOURCE_NAME}")
print(f"TB display name: {tensorboard.display_name}")

TB_RESOURCE_NAME: projects/934903580331/locations/us-central1/tensorboards/1365127248767614976
TB display name: customtest5-trainerv3-v1


## Training config

In [31]:
SEED = 1234

# =================================================
# trainconfig: GPU related
# =================================================
TF_GPU_THREAD_COUNT='8'      # '1' | '4' | '8'

# =================================================
# trainconfig: data input pipeline
# =================================================
BLOCK_LENGTH = 64            # 1, 8, 16, 32, 64
NUM_DATA_SHARDS = 4          # 2, 4, 8, 16, 32, 64
# TRAIN_PREFETCH=3

# =================================================
# trainconfig: training hparams
# =================================================
NUM_EPOCHS = 8
LEARNING_RATE = 0.01
BATCH_SIZE = 8192           # 4096, 2048, 1024, 512 

# dropout
DROPOUT_RATE = 0.33

# model size
EMBEDDING_DIM = 128
PROJECTION_DIM = 50
LAYER_SIZES = '[512,256,128]'
MAX_TOKENS = 20000     # vocab

# =================================================
# trainconfig: tensorboard
# =================================================
EMBED_FREQUENCY=0
HISTOGRAM_FREQUENCY=0
CHECKPOINT_FREQ='epoch'

In [32]:
# =================================================
# trainconfig: train & valid steps
# =================================================
train_sample_cnt = 8_205_265 # 8_205_265
valid_samples_cnt = 82_959

# validation & evaluation
VALID_FREQUENCY = 20
VALID_STEPS = valid_samples_cnt // BATCH_SIZE # 100
EPOCH_STEPS = train_sample_cnt // BATCH_SIZE

print(f"VALID_STEPS: {VALID_STEPS}")
print(f"EPOCH_STEPS: {EPOCH_STEPS}")

VALID_STEPS: 10
EPOCH_STEPS: 1001


### data sources

In [33]:
# =================================================
# trainconfig: gcs locations
# =================================================
OUTPUT_BUCKET = 'jt-tfrs-central-v3' # TODO: change this
OUTPUT_GCS_URI =f'gs://{OUTPUT_BUCKET}'

# =================================================
# trainconfig: Data sources
# =================================================
# BUCKET_DATA_DIR = 'spotify-data-regimes' 

# # data strategy: 08m
# CANDIDATE_PREFIX = 'jtv15-8m/candidates'
# TRAIN_DIR_PREFIX = 'jtv15-8m/train'     # train | train_v14
# VALID_DIR_PREFIX = 'jtv15-8m/valid'     # valid_v14

BUCKET_DATA_DIR = 'matching-engine-content'
DATA_VERSION = 'v1-0-0'

TRAIN_DIR_PREFIX = f'{DATA_VERSION}/train' # subset: valid_v9 | train_v9
VALID_DIR_PREFIX = f'{DATA_VERSION}/valid' # valid_v9 | train_v9
CANDIDATE_PREFIX = f'{DATA_VERSION}/candidates' 

### training args

In [34]:
WORKER_CMD = ["python", "two_tower_jt/task.py"]
# WORKER_CMD ["python", "-m", "trainer.task"]

WORKER_ARGS = [
    f'--project={PROJECT_ID}',
    f'--train_output_gcs_bucket={OUTPUT_BUCKET}',
    f'--train_dir={BUCKET_DATA_DIR}',
    f'--train_dir_prefix={TRAIN_DIR_PREFIX}',
    f'--valid_dir={BUCKET_DATA_DIR}',
    f'--valid_dir_prefix={VALID_DIR_PREFIX}',
    f'--candidate_file_dir={BUCKET_DATA_DIR}',
    f'--candidate_files_prefix={CANDIDATE_PREFIX}',
    f'--experiment_name={EXPERIMENT_NAME}',
    f'--experiment_run={RUN_NAME}',
    f'--num_epochs={NUM_EPOCHS}',
    f'--batch_size={BATCH_SIZE}',
    f'--embedding_dim={EMBEDDING_DIM}',
    f'--projection_dim={PROJECTION_DIM}',
    f'--layer_sizes={LAYER_SIZES}',
    f'--learning_rate={LEARNING_RATE}',
    f'--valid_frequency={VALID_FREQUENCY}',
    f'--valid_steps={VALID_STEPS}',
    f'--epoch_steps={EPOCH_STEPS}',
    f'--distribute={DISTRIBUTE_STRATEGY}',
    f'--model_version={VERSION}',
    f'--pipeline_version={VERSION}',
    f'--seed={SEED}',
    f'--max_tokens={MAX_TOKENS}',
    f'--tb_resource_name={TB_RESOURCE_NAME}',
    f'--embed_frequency={EMBED_FREQUENCY}',
    f'--hist_frequency={HISTOGRAM_FREQUENCY}',
    f'--tf_gpu_thread_count={TF_GPU_THREAD_COUNT}',
    f'--block_length={BLOCK_LENGTH}',
    f'--num_data_shards={NUM_DATA_SHARDS}',
    f'--chkpt_freq={CHECKPOINT_FREQ}',
    f'--dropout_rate={DROPOUT_RATE}',
    # uncomment these to pass value of True (bool)
    f'--cache_train',                                # caches train_dataset
    # f'--evaluate_model',                           # runs model.eval()
    # f'--write_embeddings',                         # writes embeddings index in train job
    f'--profiler',                                   # runs TB profiler
    # f'--set_jit',                                  # enables XLA
    f'--compute_batch_metrics',
    f'--use_cross_layer',
    f'--use_dropout',
]


In [35]:
from util import workerpool_specs

WORKER_POOL_SPECS = workerpool_specs.prepare_worker_pool_specs(
    image_uri=IMAGE_URI,
    args=WORKER_ARGS,
    cmd=WORKER_CMD,
    replica_count=REPLICA_COUNT,
    machine_type=WORKER_MACHINE_TYPE,
    accelerator_count=PER_MACHINE_ACCELERATOR_COUNT,
    accelerator_type=ACCELERATOR_TYPE,
    reduction_server_count=REDUCTION_SERVER_COUNT,
    reduction_server_machine_type=REDUCTION_SERVER_MACHINE_TYPE,
)

from pprint import pprint
pprint(WORKER_POOL_SPECS)

[{'container_spec': {'args': ['--project=hybrid-vertex',
                              '--train_output_gcs_bucket=jt-tfrs-central-v3',
                              '--train_dir=matching-engine-content',
                              '--train_dir_prefix=v1-0-0/train',
                              '--valid_dir=matching-engine-content',
                              '--valid_dir_prefix=v1-0-0/valid',
                              '--candidate_file_dir=matching-engine-content',
                              '--candidate_files_prefix=v1-0-0/candidates',
                              '--experiment_name=customtest5-trainerv3',
                              '--experiment_run=run-20230321-005425',
                              '--num_epochs=8',
                              '--batch_size=8192',
                              '--embedding_dim=128',
                              '--projection_dim=50',
                              '--layer_sizes=[512,256,128]',
                              '--l

### copy training package to GCS

In [36]:
BASE_OUTPUT_DIR = f'gs://{OUTPUT_BUCKET}/{EXPERIMENT_NAME}/{RUN_NAME}'

# copy training Dockerfile
!gsutil cp $REPO_DOCKER_PATH_PREFIX/cloudbuild.yaml $BASE_OUTPUT_DIR/cloudbuild.yaml
!gsutil cp $REPO_DOCKER_PATH_PREFIX/Dockerfile.tfrs $BASE_OUTPUT_DIR/Dockerfile.tfrs
!gsutil cp vocab_dict.pkl $BASE_OUTPUT_DIR/vocab_dict.pkl

# # # copy training application code
! gsutil -m cp -r $REPO_DOCKER_PATH_PREFIX/two_tower_jt/* $BASE_OUTPUT_DIR/trainer

print(f"\n Copied training package and Dockerfile to {BASE_OUTPUT_DIR}\n")

Copying file://src/cloudbuild.yaml [Content-Type=application/octet-stream]...
/ [1 files][  178.0 B/  178.0 B]                                                
Operation completed over 1 objects/178.0 B.                                      
Copying file://src/Dockerfile.tfrs [Content-Type=application/octet-stream]...
/ [1 files][  282.0 B/  282.0 B]                                                
Operation completed over 1 objects/282.0 B.                                      
Copying file://vocab_dict.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.5 MiB/ 18.5 MiB]                                                
Operation completed over 1 objects/18.5 MiB.                                     
Copying file://src/two_tower_jt/__init__.py [Content-Type=text/x-python]...
Copying file://src/two_tower_jt/data-pipeline.py [Content-Type=text/x-python]...
Copying file://src/two_tower_jt/interactive_train.py [Content-Type=text/x-python]...
Copying file://src/two_tower_jt/requirem

# submit Training job to Vertex AI

In [37]:
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    experiment=EXPERIMENT_NAME,
)

JOB_NAME = f'train-{VERSION}'

In [38]:
job = vertex_ai.CustomJob(
    display_name=JOB_NAME,
    worker_pool_specs=WORKER_POOL_SPECS,
    base_output_dir=BASE_OUTPUT_DIR,
    staging_bucket=f"{BASE_OUTPUT_DIR}/staging",
)

In [39]:
job.run(
    tensorboard=TB_RESOURCE_NAME,
    service_account=VERTEX_SA,
    restart_job_on_worker_restart=False,
    enable_web_access=True,
    sync=False,
)

## TensorBoard Profiler

Once the profiler has uploaded trace logs to `BASE_OUTPUT_DIR/logs`, we can use the in-notebook tensoborad extension to view the profiler

In [34]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf

TB_LOGS_PATH = f'{BASE_OUTPUT_DIR}/logs' # 

In [35]:
%load_ext tensorboard
# %reload_ext tensorboard

In [36]:
%tensorboard --logdir=$TB_LOGS_PATH

# Notes

In [ ]:
?job.run

Signature:
job.run(
    service_account: Union[str, NoneType] = None,
    network: Union[str, NoneType] = None,
    timeout: Union[int, NoneType] = None,
    restart_job_on_worker_restart: bool = False,
    enable_web_access: bool = False,
    tensorboard: Union[str, NoneType] = None,
    sync: bool = True,
    create_request_timeout: Union[float, NoneType] = None,
) -> None
Docstring:
Run this configured CustomJob.

Args:
    service_account (str):
        Optional. Specifies the service account for workload run-as account.
        Users submitting jobs must have act-as permission on this run-as account.
    network (str):
        Optional. The full name of the Compute Engine network to which the job
        should be peered. For example, projects/12345/global/networks/myVPC.
        Private services access must already be configured for the network.
        If left unspecified, the job is not peered with any network.
    timeout (int):
        The maximum job running time in seconds.